# Security Support Provider (SSP) added to LSA configuration
Detects the addition of a SSP to the registry. Upon a reboot or API call, SSP DLLs gain access to encrypted and plaintext passwords stored in Windows.

## Rule Content
```
- title: Security Support Provider (SSP) added to LSA configuration
  id: eeb30123-9fbd-4ee8-aaa0-2e545bbed6dc
  status: experimental
  description: Detects the addition of a SSP to the registry. Upon a reboot or API
    call, SSP DLLs gain access to encrypted and plaintext passwords stored in Windows.
  references:
  - https://attack.mitre.org/techniques/T1101/
  - https://powersploit.readthedocs.io/en/latest/Persistence/Install-SSP/
  tags:
  - attack.persistence
  - attack.t1011
  author: iwillkeepwatch
  date: 2019/01/18
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection_registry:
      EventID: 13
      TargetObject:
      - HKLM\System\CurrentControlSet\Control\Lsa\Security Packages
      - HKLM\System\CurrentControlSet\Control\Lsa\OSConfig\Security Packages
    exclusion_images:
    - Image: C:\Windows\system32\msiexec.exe
    - Image: C:\Windows\syswow64\MsiExec.exe
    condition: selection_registry and not exclusion_images
  falsepositives:
  - Unlikely
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"13" AND registry_key_path:("HKLM\\System\\CurrentControlSet\\Control\\Lsa\\Security\ Packages" OR "HKLM\\System\\CurrentControlSet\\Control\\Lsa\\OSConfig\\Security\ Packages")) AND (NOT (process_path:"C\:\\Windows\\system32\\msiexec.exe" OR process_path:"C\:\\Windows\\syswow64\\MsiExec.exe")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()